In [16]:
import numpy as np
import matplotlib.pyplot as plt
import casadi
import scipy.interpolate
from autoware_vehicle_adaptor.param import parameters

# Set Parameters

In [17]:
# constraint
max_velocity = 11.1 # 11m/s=40km/h # max velocity limit [m/s]
max_lateral_accel = 0.8 # max lateral acceleration limit [m/ss] # https://github.com/autowarefoundation/autoware.universe/blob/98c24b824c68605c28513de47a36aaf5a4a89817/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L15
min_acc = -1.0
max_acc = 1.0
min_jerk = -0.5
max_jerk = 0.5
max_steering_angle_rate = 40.0 # maximum steering angle rate [degree/s] # https://github.com/autowarefoundation/autoware.universe/blob/98c24b824c68605c28513de47a36aaf5a4a89817/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L22C5-L22C86

# vehicle parameter
wheel_base = parameters.wheel_base

# path parameter
dpos = 0.01
dsteer_dpos_max = 0.04
speed_transition_step = 500
stopping_time = 10.0

# Define curve path generation functions

In [ ]:
def get_left_turn_path():
    x = 0.0
    y = 0.0
    yaw = 0.0
    steer = 0.0
    temp_path = []
    temp_path.append([x,y,yaw,steer])
    while True:
        x += dpos * np.cos(yaw)
        y += dpos * np.sin(yaw)
        yaw += dpos * np.tan(steer) / wheel_base
        steer += dsteer_dpos_max * dpos
        temp_path.append([x,y,yaw,steer])
        if yaw >= np.pi / 4:
            break
    if np.pi / 4 - temp_path[-2][2] < temp_path[-1][2] - np.pi / 4:
        temp_path.pop()
    states_prev = temp_path[-1]
    x_prev = states_prev[0]
    y_prev = states_prev[1]
    intercept = (x_prev + 0.5 * dpos * np.cos(np.pi / 4)) + (y_prev + 0.5 * dpos * np.sin(np.pi / 4))
    temp_path_head = temp_path.copy()    
    for i in range(1, len(temp_path) + 1):
        states_inverse = temp_path_head[- i]
        x_inverse = states_inverse[0]
        y_inverse = states_inverse[1]
        yaw_inverse = states_inverse[2]
        steer_inverse = states_inverse[3]
        x = -y_inverse + intercept
        y = -x_inverse + intercept
        yaw = np.pi / 2 - yaw_inverse
        steer = steer_inverse
        temp_path.append([x,y,yaw,steer])
    return np.array(temp_path)
def get_right_turn_path():
    temp_path = []
    left_turn_path = get_left_turn_path()
    for states in left_turn_path:
        x_left = states[0]
        y_left = states[1]
        yaw_left = states[2]
        steer_left = states[3]
        temp_path.append([-x_left, y_left, np.pi - yaw_left, - steer_left])
    return np.array(temp_path)
left_turn_path = get_left_turn_path()
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5), tight_layout=True)
fig.suptitle("left curve path plots")
axes[0].plot(left_turn_path[:,0],left_turn_path[:,1])
axes[0].plot(left_turn_path[0,0],left_turn_path[0,0], 'o', label="start point")
axes[0].legend()
axes[0].set_aspect('equal')
axes[0].set_xlabel('x_position [m]')
axes[0].set_ylabel('y_position [m]')
axes[0].set_title('path position')
axes[1].plot(dpos * np.arange(left_turn_path.shape[0]),left_turn_path[:,2])
axes[1].set_xlabel('travel distance [m]')
axes[1].set_ylabel('yaw angle [rad]')
axes[1].set_title('yaw angle')
axes[2].plot(dpos * np.arange(left_turn_path.shape[0]),left_turn_path[:,3])
axes[2].set_xlabel('travel distance [m]')
axes[2].set_ylabel('steer angle [rad]')
axes[2].set_title('steer angle')
plt.show()

right_turn_path = get_right_turn_path()
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5), tight_layout=True)
fig.suptitle("right curve path plots")
axes[0].plot(right_turn_path[:,0],right_turn_path[:,1])
axes[0].plot(right_turn_path[0,0],right_turn_path[0,0], 'o', label="start point")
axes[0].legend()
axes[0].set_aspect('equal')
axes[0].set_xlabel('x_position [m]')
axes[0].set_ylabel('y_position [m]')
axes[0].set_title('path position')
axes[1].plot(dpos * np.arange(right_turn_path.shape[0]),right_turn_path[:,2])
axes[1].set_xlabel('travel distance [m]')
axes[1].set_ylabel('yaw angle [rad]')
axes[1].set_title('yaw angle')
axes[2].plot(dpos * np.arange(right_turn_path.shape[0]),right_turn_path[:,3])
axes[2].set_xlabel('travel distance [m]')
axes[2].set_ylabel('steer angle [rad]')
axes[2].set_title('steer angle')
plt.show()

# Define total path generation function

In [19]:
def get_total_path(x_length,y_length):
    left_turn_path = get_left_turn_path()
    right_turn_path = get_right_turn_path()
    x_num = int(x_length // dpos)
    y_num = int(y_length // dpos)
    x = 0.0
    y = 0.0
    yaw = 0.0
    steer = 0.0
    total_path = []
    total_path.append(np.array([x,y,yaw,steer]))
    for i in range(2 * x_num + 2*int((left_turn_path[-1,0] - left_turn_path[0,0])//dpos)):
        x += dpos
        total_path.append(np.array([x,y,yaw,steer]))
    first_position = total_path[-1]
    for i in range(1,len(left_turn_path)):
        states = left_turn_path[i]
        x = states[0] + first_position[0]
        y = states[1] + first_position[1]
        yaw = states[2]
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    second_position = total_path[-1]
    for i in range(y_num):
        x = second_position[0]
        y = second_position[1] + (i+1)*dpos
        yaw = second_position[2]
        steer = second_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    third_position = total_path[-1]
    for i in range(1,len(left_turn_path)):
        states = left_turn_path[i]
        x = - states[1] + third_position[0]
        y = states[0] + third_position[1]
        yaw = states[2] + np.pi / 2
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))       
    forth_position = total_path[-1]
    for i in range(x_num):
        x = forth_position[0] - (i+1)*dpos
        y = forth_position[1]
        yaw = forth_position[2]
        steer = forth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    fifth_position = total_path[-1]
    stopping_target = len(total_path) - 1
    for i in range(1,len(left_turn_path)):
        states = left_turn_path[i]
        x = - states[0] + fifth_position[0]
        y = - states[1] + fifth_position[1]
        yaw = states[2] + np.pi
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    sixth_position = total_path[-1]
    for i in range(y_num):
        x = sixth_position[0]
        y = sixth_position[1] - (i+1)*dpos
        yaw = sixth_position[2]
        steer = sixth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    seventh_position = total_path[-1]
    for i in range(1,len(right_turn_path)):
        states = right_turn_path[i]
        x = - states[1] + seventh_position[0]
        y = states[0] + seventh_position[1]
        yaw = states[2] + np.pi / 2
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    eighth_position = total_path[-1]
    for i in range(x_num):
        x = eighth_position[0] - (i+1)*dpos
        y = eighth_position[1]
        yaw = eighth_position[2]
        steer = eighth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    ninth_position = total_path[-1]
    for i in range(1,len(right_turn_path)):
        states = right_turn_path[i]
        x = states[0] + ninth_position[0]
        y = states[1] + ninth_position[1]
        yaw = states[2]
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    tenth_position = total_path[-1]
    for i in range(y_num):
        x = tenth_position[0]
        y = tenth_position[1] + (i+1)*dpos
        yaw = tenth_position[2]
        steer = tenth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))
    eleventh_position = total_path[-1]
    for i in range(1,len(right_turn_path)):
        states = right_turn_path[i]
        x = states[1] + eleventh_position[0]
        y = - states[0] + eleventh_position[1]
        yaw = states[2] - np.pi / 2
        steer = states[3]
        total_path.append(np.array([x,y,yaw,steer]))
    twelfth_position = total_path[-1]
    for i in range(2 * x_num + 2 * int((left_turn_path[-1,0] - left_turn_path[0,0])//dpos)):
        x = twelfth_position[0] + (i+1)*dpos
        y = twelfth_position[1]
        yaw = twelfth_position[2]
        steer = twelfth_position[3]
        total_path.append(np.array([x,y,yaw,steer]))   
    return np.array(total_path), stopping_target

# Generate total path

In [ ]:
x_length = 100.0 # Distance in x-direction from the completion of a curve to the next curve
y_length = 30.0 # Distance in y-direction from the completion of a curve to the next curve
total_path, stopping_target = get_total_path(x_length,y_length)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5), tight_layout=True)
fig.suptitle("total path plots")
axes[0].plot(total_path[:,0], total_path[:,1])
axes[0].plot(total_path[0,0], total_path[0,1], 'o', label="start point")
axes[0].plot(total_path[-1,0], total_path[-1,1], 'o', label="end point")
axes[0].plot(total_path[stopping_target,0], total_path[stopping_target,1], 'o', label="stop point")
axes[0].legend()
axes[0].set_xlabel('x_position [m]')
axes[0].set_ylabel('y_position [m]')
axes[0].set_title('path position')
axes[1].plot(dpos * np.arange(total_path.shape[0]),total_path[:,2])
axes[1].set_xlabel('travel distance [m]')
axes[1].set_ylabel('yaw angle [rad]')
axes[1].set_title('yaw angle')
axes[2].plot(dpos * np.arange(total_path.shape[0]),total_path[:,3])
axes[2].set_xlabel('travel distance [m]')
axes[2].set_ylabel('steer angle [rad]')
axes[2].set_title('steer angle')
plt.show()

# Initial velocity limit

In [ ]:
velocity_limit = np.array([max_velocity]*len(total_path))
plt.plot(dpos * np.arange(total_path.shape[0]), velocity_limit,label="external velocity limit")
plt.xlabel("travel distance [m]",fontsize=15)
plt.ylabel("longitudinal velocity [m/s]",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.legend()
plt.show()

# Stopping Constraint

In [ ]:
applied_velocity_limit = []

for i in range(len(total_path)):
    if stopping_target - speed_transition_step <= i <= stopping_target + speed_transition_step:
        applied_velocity_limit.append(max_velocity * np.abs(i - stopping_target)/speed_transition_step)
    elif i <= speed_transition_step:
        applied_velocity_limit.append(max_velocity * i/speed_transition_step)
    elif i >= len(total_path) - speed_transition_step -1:
        applied_velocity_limit.append(max_velocity * (len(total_path) - i -1)/speed_transition_step)
    else:
        applied_velocity_limit.append(np.inf)
applied_velocity_limit = np.array(applied_velocity_limit)
plt.plot(dpos * np.arange(total_path.shape[0]),velocity_limit, label="current velocity limit")
plt.plot(dpos * np.arange(total_path.shape[0]),applied_velocity_limit, label="stopping velocity limit")
plt.legend()
plt.xlabel("travel distance [m]",fontsize=15)
plt.ylabel("longitudinal velocity [m/s]",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.show()
velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Applying lateral acceleration limit
$$
    max\_lateral\_accel \ge r \left(\frac{dyaw}{dt}\right)^2 = \frac{v^2}{r}
$$
$$
    \sqrt{r\times max\_lateral\_accel} \ge v
$$

In [ ]:
r_trajectory = wheel_base/np.abs(np.tan(total_path[:,3])+1e-16)
applied_velocity_limit = np.sqrt(max_lateral_accel*r_trajectory )

plt.plot(dpos * np.arange(total_path.shape[0]),velocity_limit, label="current velocity limit")
plt.plot(dpos * np.arange(total_path.shape[0]),applied_velocity_limit, label="lateral acceleration limit")

plt.legend()
plt.xlabel("travel distance [m]",fontsize=15)
plt.ylabel("longitudinal velocity [m/s]",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.show()
velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Applying steering rate limit
$$
    steer\_rate\_limit \ge \left|\frac{dsteer}{dt}\right| = \left|\frac{dpos}{dt} \frac{dsteer}{dpos}\right| = v\left|\frac{dsteer}{dpos}\right|
$$
$$
    \frac{steer\_rate\_limit}{\left|\frac{dsteer}{dpos}\right|} \ge v
$$

In [ ]:
dsteer_dpos = (total_path[1:,3]-total_path[:-1,3])/dpos
dsteer_dpos = np.hstack([dsteer_dpos,dsteer_dpos[-1]])
applied_velocity_limit = (max_steering_angle_rate*np.pi/180.0) / np.abs(1e-16+dsteer_dpos)
print(applied_velocity_limit.shape)
plt.plot(dpos * np.arange(total_path.shape[0]),velocity_limit, label="current velocity limit")
plt.plot(dpos * np.arange(total_path.shape[0]),applied_velocity_limit, label="steering rate limit")
plt.ylim([-1,max_velocity+1])
plt.xlabel("travel distance [m]",fontsize=15)
plt.ylabel("longitudinal velocity [m/s]",fontsize=15)
plt.legend()
plt.show()
velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Final velocity limit

In [ ]:
plt.plot(dpos * np.arange(total_path.shape[0]),velocity_limit, label="final velocity limit")
plt.ylim([-1,max_velocity+1])
plt.xlabel("travel distance [m]",fontsize=15)
plt.ylabel("longitudinal velocity [m/s]",fontsize=15)
plt.legend()
plt.show()

# velocity smoothing optimization (simplified version)
$$
    \begin{matrix}
    \min & -vel^2 + \lambda_1 * (d vel/ d pos)^2 + \lambda_2 * (d^2 vel/ d pos^2)^2 \\
    \mbox{subject to } & 0 \le vel \le vel_{limit} \\ 
    & acc_{min}\le acc \le acc_{max} \\
    & jerk_{min}\le jerk \le jerk_{max}
    \end{matrix}
$$

In [ ]:
lambda_1 = 1e-4
lambda_2 = 1e-8
n =len(velocity_limit)
v = casadi.SX.sym('v', n)
w = [v]
lbw = [0]*n
ubw = velocity_limit.tolist()


g = []
lbg = []
ubg = []
J = - (v.T @ v)
print("check 0")
for i in range(n-1):
    a = 0.5 * (v[i+1] * v[i+1] - v[i] * v[i])/dpos
    J += lambda_1 * (v[i+1] - v[i]) * (v[i+1] - v[i]) / (dpos * dpos)
    g.append(a)
    lbg.append(min_acc)
    ubg.append(max_acc)
for i in range(n-2):
    #a1 = 0.5 * (v[i+1] * v[i+1]- v[i] * v[i])/dpos    
    #a2 = 0.5 * (v[i+2] * v[i+2] - v[i+1] * v[i+1])/dpos
    jerk = 0.25 * (v[i+2] * v[i+2] - 2 * v[i+1] * v[i+1] + v[i] * v[i]) * (v[i] + v[i+1]) / (dpos * dpos)
    J += lambda_2 * (v[i+2] - 2 * v[i+1] + v[i]) * (v[i+2] - 2 * v[i+1] + v[i]) / (dpos * dpos * dpos * dpos)
    g.append(jerk)
    lbg.append(min_jerk)
    ubg.append(max_jerk)

print("check 1")
nlp = {'f': J, 'x': casadi.vertcat(*w), 'g': casadi.vertcat(*g)} 
print("check 2")
solver = casadi.nlpsol('S','ipopt',nlp)
print("check 3")
sol = solver(x0=([0]*n), lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)

# Smoothed velocity

In [ ]:
v_sol = sol["x"].toarray().flatten()
v_sol[0] = 0.0
v_sol[stopping_target] = 0.0
v_sol[-1] = 0.0
for i in range(min(int(1.0 / dpos), stopping_target//2, (total_path.shape[0] - stopping_target)//2)):
    v_sol[i] = min(max(v_sol[i], np.sqrt(2 * i * dpos * 0.1 *max_acc)), v_sol[int(1.0 / dpos)])
    v_sol[-i - 1] = min(max(v_sol[-i - 1], np.sqrt(2 * i * dpos * 0.1 *max_acc)), v_sol[-int(1.0 / dpos) - 1])
    v_sol[stopping_target - i] = min(max(v_sol[stopping_target - i], np.sqrt(2 * i * dpos * 0.1 *max_acc)), v_sol[stopping_target - int(1.0 / dpos)])
    v_sol[stopping_target + i] = min(max(v_sol[stopping_target + i], np.sqrt(2 * i * dpos * 0.1 *max_acc)), v_sol[stopping_target + int(1.0 / dpos)])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 5), tight_layout=True)
fig.suptitle("smoothed velocity plots")
axes[0].plot(dpos * np.arange(total_path.shape[0]),velocity_limit, label="final velocity limit")
axes[0].plot(dpos * np.arange(total_path.shape[0]),v_sol, label="smoothing result")
axes[0].set_ylim([-1,max_velocity+1])
axes[0].set_xlabel("travel distance [m]")
axes[0].set_ylabel("longitudinal velocity [m/s]")
axes[1].set_title('total velocity path')
axes[1].plot(dpos * np.arange(total_path.shape[0]),v_sol, label="smoothing result")
axes[1].set_xlim([dpos * (stopping_target - 10), dpos * (stopping_target + 10)])
axes[1].set_ylim([-0.2,0.2])
axes[1].set_xlabel("travel distance [m]")
axes[1].set_ylabel("longitudinal velocity [m/s]")
axes[1].set_title('velocity path around stopping point')
plt.show()

# Total path before interpolation

In [ ]:
dt_path = 2 * dpos/(v_sol[1:] + v_sol[:-1])
timestamp = []
xy_path = []
v_path = []
yaw_path = []
steer_path = []
t = 0
for i in range(len(dt_path)):
    timestamp.append(1.0*t)
    xy_path.append(total_path[i,[0,1]])
    v_path.append(v_sol[i])
    yaw_path.append(total_path[i,2])
    steer_path.append(total_path[i,-1])
    if i == stopping_target or i == len(dt_path) - 1:
        t += stopping_time
        timestamp.append(1.0 * t)
        xy_path.append(total_path[i,[0,1]])
        v_path.append(v_sol[i])
        yaw_path.append(total_path[i,2])
        steer_path.append(total_path[i,-1])
    t += dt_path[i]
timestamp = np.array(timestamp)
xy_path = np.array(xy_path)
v_path = np.array(v_path)
yaw_path = np.array(yaw_path)
steer_path = np.array(steer_path)

a_path = (v_path[1:] - v_path[:-1]) / (timestamp[1:] - timestamp[:-1])
a_path = np.hstack([a_path, a_path[-1]])
jerk_path = (a_path[1:] - a_path[:-1]) / (timestamp[1:] - timestamp[:-1])
jerk_path = np.hstack([jerk_path, jerk_path[-1]])
dsteer_path = (steer_path[1:] - steer_path[:-1]) / (timestamp[1:] - timestamp[:-1])
dsteer_path = np.hstack([dsteer_path,dsteer_path[-1]])

trajectory_data=np.zeros((timestamp.shape[0],9))
trajectory_data[:,0] = timestamp
trajectory_data[:,[1,2]] = xy_path 
trajectory_data[:,3] = v_path
trajectory_data[:,4] = yaw_path
trajectory_data[:,5] = a_path
trajectory_data[:,6] = steer_path
trajectory_data[:,7] = jerk_path
trajectory_data[:,8] = dsteer_path
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(18, 5), tight_layout=True)
fig.suptitle("total path before interpolation")
title_name_list = ["x", "y", "vel", "yaw", "acc", "steer", "jerk", "dsteer"]
unit_name_list = ["[m]", "[m]", "[m/s]", "[rad]", "[m/s^2]", "[rad]", "[m/s^3]", "[rad/s]"]
for i in range(8):
    axes[int(i/4),i%4].plot(trajectory_data[:,0],trajectory_data[:,i+1])
    axes[int(i/4),i%4].set_title(title_name_list[i] + "_path")
    axes[int(i/4),i%4].set_xlabel("Time [s]")
    axes[int(i/4),i%4].set_ylabel(title_name_list[i] + " " + unit_name_list[i])
plt.show()

# Total path after interpolation

In [ ]:
timestamp_interp = []
diff_step_for_jerk = 20
min_dt = 0.05
for i in range(timestamp.shape[0]):
    timestamp_interp.append(timestamp[i])
    if i == timestamp.shape[0] - 1:
        continue
    else:
        while timestamp_interp[-1] + min_dt < timestamp[i+1]:
            timestamp_interp.append(timestamp_interp[-1] + min_dt)
timestamp_interp = np.array(timestamp_interp)
trajectory_interpolator_list = [
            scipy.interpolate.interp1d(trajectory_data[:, 0], trajectory_data[:, 1 + i])
            for i in range(trajectory_data.shape[1] - 1)
        ]
trajectory_interp = np.zeros((timestamp_interp.shape[0],9))
trajectory_interp[:,0] = timestamp_interp
for i in range(8):
    trajectory_interp[:,i+1] = trajectory_interpolator_list[i](timestamp_interp)
trajectory_interp[:-1,5] = (trajectory_interp[1:,3] - trajectory_interp[:-1,3])/(timestamp_interp[1:] - timestamp_interp[:-1])
trajectory_interp[-1,5] = trajectory_interp[-2,5]
trajectory_interp[diff_step_for_jerk:-diff_step_for_jerk,7] = (trajectory_interp[2 * diff_step_for_jerk:,5] - trajectory_interp[:- 2 * diff_step_for_jerk,5])/(timestamp_interp[2 * diff_step_for_jerk:] - timestamp_interp[:- 2 * diff_step_for_jerk])
trajectory_interp[:-1,8] = (trajectory_interp[1:,6] - trajectory_interp[:-1,6])/(timestamp_interp[1:] - timestamp_interp[:-1])
trajectory_interp[-1,8] = trajectory_interp[-2,6]

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(18, 5), tight_layout=True)
fig.suptitle("total path after interpolation")
for i in range(8):
    axes[int(i/4),i%4].plot(trajectory_interp[:,0],trajectory_interp[:,i+1])
    axes[int(i/4),i%4].set_title(title_name_list[i] + "_path")
    axes[int(i/4),i%4].set_xlabel("Time [s]")
    axes[int(i/4),i%4].set_ylabel(title_name_list[i] + " " + unit_name_list[i])
plt.show()

# Save data as csv

In [30]:
np.savetxt("mpc_figure_eight_course_data.csv",trajectory_interp,delimiter=',')